#### Player id lookup notbook  
In this notbook I create a look-up table that maps players' collegian id to NBA id.  
The two ids are different and it's not possible to recreate these ids reliably.  
However each NBA player who has played in College will have a link in their profile page which leads into their College profile.  
To get the two id I do the following:  
1. Find a page where all NBA players for the season will be listed. The easiest I found is the season stats for each year. (Note that if a player did not play in that year due to injury or something else, they will not be on this list.)  
2. From this list extract each players' profile page link
3. Go to each profile page and locate their Collegian link, part of that link will have their Collegian player id. 
- for the first year, go through every players' profile
- for the following years, just find new players/players who did not play in the previous year
4. Store that info in a seperate table which is then downloaded as CSV. 

In [134]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from lxml import etree, html
import os
import numpy as np
from datetime import datetime
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_colwidth", 200)

**If updating the player_id_lookup table, first load the current table**

In [402]:
player_id_lookup = pd.read_csv('02_database/player_id_lookup.csv')

**player_id_lookup clean up**

In [398]:
player_id_lookup['nba_player_id'] = player_id_lookup.nba_link.apply(lambda row: row[::-1]).str.strip().str.split('/',expand = True, n = 1)[0].apply(lambda row: row[::-1]).str.split('.',expand = True)[0]
player_id_lookup['cbb_player_id'] = player_id_lookup.cbb_link.apply(lambda row: row[::-1] if(pd.notnull(row)) else np.nan).str.strip().str.split('/',expand = True, n = 1)[0].apply(lambda row: row[::-1] if(pd.notnull(row)) else np.nan).str.split('.',expand = True)[0]
player_id_lookup.head(2)

,name,cbb_link,nba_link,nba_player_id,cbb_player_id
0,LeBron James,NaN,https://www.basketball-reference.com/players/j/jamesle01.html,jamesle01,NaN
1,Derrick Rose,https://www.sports-reference.com/cbb/players/derrick-rose-1.html,https://www.basketball-reference.com/players/r/rosede01.html,rosede01,derrick-rose-1


In [399]:
player_id_lookup.to_csv('02_database/player_id_lookup.csv', index = False)

## START FROM HERE

In [416]:
# find new players in nba_player table for which you do not have a row in look-up table
# look up cbb id for those players only
player_id_lookup = pd.read_csv('02_database/player_id_lookup.csv')
nba_player = pd.read_csv('02_database/nba_player.csv')

## Updating player_id_lookup table with new players 
* after updating nba_player table, check which nba players are not in the `player_id_lookup` table
* for each of those players, get their cbb link

In [437]:
# Find new player is nba_player table that are not in player_id_lookup table
new_players = nba_player.merge(player_id_lookup, how = 'left', right_on = 'nba_player_id',left_on = 'player_id',indicator = True)
new_players = new_players.loc[(new_players['_merge'] != 'both'),['player_id','name_x']]
new_players['link'] = new_players.player_id.apply(lambda x: 'https://www.basketball-reference.com/players/' + x[0] + '/' + x + '.html')
new_players.rename(columns = {'name_x':'name'},inplace = True)
new_players.head(2)

,player_id,name,link
3,fournev01,Evan Fournier,https://www.basketball-reference.com/players/f/fournev01.html
11,millequ01,Quincy Miller,https://www.basketball-reference.com/players/m/millequ01.html


**For each new player - get thier cbb link**

In [438]:
# convert df to tuples
link_player_tpls = [tuple(x) for x in new_players.loc[:,['name','link']].to_numpy()] # only add in new players

name_lst = []
cbb_link = []
nba_link = []
for name, link in link_player_tpls:
    url = link
    page = requests.get(url)
    soup = BeautifulSoup(page.text,'lxml')
    cbb_info = soup.find_all('a',{'href':re.compile('https://www.sports-reference.com/cbb/players')})#[0].attrs['href']
    if len(cbb_info) > 0:
        name_lst.append(name)
        cbb_link.append(cbb_info[0].attrs['href'])
        nba_link.append(link)
        # print(name, 'college')
        # print(cbb_info[0].attrs['href'])
    else:
        name_lst.append(name)
        cbb_link.append(np.nan)
        nba_link.append(link)
# store results in df        
df = pd.DataFrame({'name':name_lst,'cbb_link':cbb_link,'nba_link':nba_link})
# apply clean up
df['nba_player_id'] = df.nba_link.apply(lambda row: row[::-1]).str.strip().str.split('/',expand = True, n = 1)[0].apply(lambda row: row[::-1]).str.split('.',expand = True)[0]
df['cbb_player_id'] = df.cbb_link.apply(lambda row: row[::-1] if(pd.notnull(row)) else np.nan).str.strip().str.split('/',expand = True, n = 1)[0].apply(lambda row: row[::-1] if(pd.notnull(row)) else np.nan).str.split('.',expand = True)[0]


In [443]:
# merge with the current table
player_id_lookup = pd.concat([player_id_lookup,df],axis = 0)

In [448]:
player_id_lookup.to_csv('02_database/player_id_lookup.csv',index = False)

# DON'T NEED THE REST

In [146]:
years = [2009,2010,2011,2012]
url_root = 'https://www.basketball-reference.com/leagues/NBA_'  
url_end='_per_game.html'
link_player = pd.DataFrame()
for year in years:
    print('working on year: ',year, datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
    url = url_root+str(year)+url_end
    page = requests.get(url)
    soup = BeautifulSoup(page.text,'lxml')

    # get player name and link for their profile
    link_root = 'https://www.basketball-reference.com'
    player_link=[]
    player_name = []
    year_roster = []
    full_table_tags = soup.find_all('a',{'href':re.compile('/players/')})
    for i in full_table_tags[1:]:
        player_link.append('https://www.basketball-reference.com'+i.attrs['href'])
        player_name.append(i.text)
        year_roster.append(year)
        
    df = pd.DataFrame({'name':player_name,'link':player_link,'year':year_roster}).drop_duplicates()
    df.drop(df[df.name == 'Players'].index,inplace = True) # remove Players as a name
    link_player = pd.concat([df,link_player],axis = 0)

working on year:  2009 13/08/2022 21:12:03
working on year:  2010 13/08/2022 21:12:05
working on year:  2011 13/08/2022 21:12:07
working on year:  2012 13/08/2022 21:12:08


In [188]:
# Using 2009 as starting table, find all new players from 2010,2011,2012 years
df = link_player.merge(player_id_lookup, how = 'left', left_on = 'link',right_on = 'nba_link')

new_players = df.loc[df.name_y.isna(),['name_x','link','year']]
new_players.rename(columns = {'name_x':'name'},inplace = True)

**go into player profile**

In [190]:
# convert df to tuples
link_player_tpls = [tuple(x) for x in new_players.loc[:,['name','link']].to_numpy()] # only add in new players
# link_player_tpls = [tuple(x) for x in link_player[link_player.year==2009].loc[:,['name','link']].to_numpy()] # 2009 only as base line
name_lst = []
cbb_link = []
nba_link = []
for name, link in link_player_tpls:
    url = link
    page = requests.get(url)
    soup = BeautifulSoup(page.text,'lxml')
    cbb_info = soup.find_all('a',{'href':re.compile('https://www.sports-reference.com/cbb/players')})#[0].attrs['href']
    if len(cbb_info) > 0:
        name_lst.append(name)
        cbb_link.append(cbb_info[0].attrs['href'])
        nba_link.append(link)
        # print(name, 'college')
        # print(cbb_info[0].attrs['href'])
    else:
        name_lst.append(name)
        cbb_link.append(np.nan)
        nba_link.append(link)
# store results in df        
df = pd.DataFrame({'name':name_lst,'cbb_link':cbb_link,'nba_link':nba_link})

In [196]:
# append results to the current lookup table
player_id_lookup = pd.concat([player_id_lookup,df])

In [198]:
# save the base table as csv
player_id_lookup.to_csv('player_id_lookup_2009_base.csv',index = False)

#### Clean up the lookup table
- remove duplicate names where the names are L.James instead of LeBron James. Should be just a small handfull
- create a sperate columns for the two IDs. Currently they are still in the links

In [199]:
player_id_lookup.head()

,name,cbb_link,nba_link
0,LeBron James,NaN,https://www.basketball-reference.com/players/j/jamesle01.html
1,Derrick Rose,https://www.sports-reference.com/cbb/players/derrick-rose-1.html,https://www.basketball-reference.com/players/r/rosede01.html
2,Dwyane Wade,https://www.sports-reference.com/cbb/players/dwyane-wade-1.html,https://www.basketball-reference.com/players/w/wadedw01.html
3,Dwight Howard,NaN,https://www.basketball-reference.com/players/h/howardw01.html
4,Chris Paul,https://www.sports-reference.com/cbb/players/chris-paul-1.html,https://www.basketball-reference.com/players/p/paulch01.html


## Keep for later

In [130]:
# duplicate names in link_player
# only a few, can get rid of them later on
x = link_player.groupby(['link'])['name'].count().sort_values(ascending = False)
link_player[link_player.link.isin(list(x[x>1].index))]

,name,link
0,LeBron James,https://www.basketball-reference.com/players/j/jamesle01.html
2,Kevin Durant,https://www.basketball-reference.com/players/d/duranke01.html
126,Stephen Curry,https://www.basketball-reference.com/players/c/curryst01.html
217,James Harden,https://www.basketball-reference.com/players/h/hardeja01.html
430,Chris Paul,https://www.basketball-reference.com/players/p/paulch01.html
585,L. James,https://www.basketball-reference.com/players/j/jamesle01.html
586,K. Durant,https://www.basketball-reference.com/players/d/duranke01.html
588,J. Harden,https://www.basketball-reference.com/players/h/hardeja01.html
589,S. Curry,https://www.basketball-reference.com/players/c/curryst01.html
601,C. Paul,https://www.basketball-reference.com/players/p/paulch01.html


## TESTS

**NCAA Loop - get all player ids**

In [ ]:
# from sportsipy.ncaab.teams import Teams

# years = [2010,2011,2012]
# for year in years:
#     teams = Teams(year = year)
#     team_lst = [team for team in teams] # all teams in a list allows me to select just a subset of teams for testing

#     df = pd.DataFrame()

#     for i, team in enumerate(team_lst):
#         print(str(year)+':'+str(i)+' of '+str(len(team_lst))+':'+ team.name +'start at: '+datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
#         roster = team.roster  # Gets each team's roster
#         for player in roster.players:
#             player_df = pd.concat([player.dataframe.reset_index(),pd.DataFrame({'name':[player.name]*player.dataframe.shape[0]})],axis = 1) # add name to df
#             df = pd.concat([df,player_df],axis = 0)
#     # create csv
#     df.to_csv('cbb_'+str(year)+'.csv',index = False)

**Get awards for each player**  
first need to get all the player IDs from the code above

In [290]:
# url_root = 'https://www.sports-reference.com/cbb/players/'

# # list of all player id to loop through
# player_id = ['temetrius-morant-1','aaron-douglas-2']

# # empty link to hold results
# # player_id::award::award category on website
# awards = []

# for id_ in player_id:
#     url = url_root+id_+'.html'
#     response = requests.get(url)
#     html = response.text
#     html = html.replace('<!--', '').replace('-->', '')
#     soup = BeautifulSoup(html, 'lxml')
#     leaderboard = soup.find_all('div', {'id':re.compile('leaderboard_')})
    
#     # check if player has awards
#     if len(leaderboard)>0:
#         for l in leaderboard[1:]:
#             for i in l.find_all('td',{'class':'single'}):
#                 awards.append(id_ + '::' +i.text +'::' + l.find('caption',{'class':'poptip'}).text)
#     else:
#         awards.append(id_ + '::' +'no_awards' +'::' +'no_awards')

In [293]:
# clean this up
pd.Series(awards).head(4)

0                  temetrius-morant-1::2018-19 All-OVC - 1st Team::Awards
1    temetrius-morant-1::2018-19 Consensus All-America - 1st Team::Awards
2            temetrius-morant-1::2018-19 Naismith Award Finalists::Awards
3        temetrius-morant-1::2018-19 Naismith Award Semifinalists::Awards
dtype: object

In [ ]:
#### DONE, DONT NEED CODE BELOW!!

In [275]:
### awards from players' profile
url = 'https://www.sports-reference.com/cbb/players/temetrius-morant-1.html'
response = requests.get(url)
html = response.text
html = html.replace('<!--', '').replace('-->', '')

soup = BeautifulSoup(html, 'lxml')
leaderboard = soup.find_all('div', {'id':re.compile('leaderboard_')})

# all awards and name of award group separated by ::
awards = []
for l in leaderboard[1:]:
    for i in l.find_all('td',{'class':'single'}):
        awards.append(i.text +'::' + l.find('caption',{'class':'poptip'}).text)

        
        
# awards_sub_rect = []
# for i in awards_rect:
#     awards_sub_rect.append(i)

# award_lst = []
# award_cat = []

# for i in awards_html:
#     for a in i:
#         a.find_all()

In [202]:
url = 'https://www.sports-reference.com/cbb/players/kevin-durant-1.html'
page = requests.get(url)
soup = BeautifulSoup(page.text,'lxml')

In [203]:
url = 'https://www.sports-reference.com/cbb/players/temetrius-morant-1.html'
response = requests.get(url)
html = response.text
html = html.replace('<!--', '').replace('-->', '')

soup = BeautifulSoup(html, 'html.parser')
leaderboard_pts = soup.find('div', {'id':'leaderboard_pts'})

In [204]:
leaderboard_pts

<div class="data_grid_box" id="leaderboard_pts">
<table class="no_columns">
<caption class="poptip" data-tip="Points">Points</caption> <tr class=""> <td class="single"><a href="/cbb/conferences/ovc/2018-leaders.html">2017-18</a> <a href="/cbb/conferences/ovc/2018-leaders.html">OVC</a> 405 (18th)</td>
</tr>
<tr class=""> <td class="single"><a href="/cbb/seasons/2019-leaders.html">2018-19</a> <a href="/cbb/seasons/2019-leaders.html">NCAA</a> 808 (9th)</td>
</tr>
<tr class="first_place"> <td class="single"><a href="/cbb/conferences/ovc/2019-leaders.html">2018-19</a> <a href="/cbb/conferences/ovc/2019-leaders.html">OVC</a> 808 (1st)</td>
</tr>
</table>
<button onclick="vjs_addClass(document.getElementById('leaderboard_pts'),'show_all')">3 Seasons in Top 20</button>
</div>

In [203]:
df = pd.read_html(str(leaderboard_pts))[0]
df

,0
0,2017-18 OVC 405 (18th)
1,2018-19 NCAA 808 (9th)
2,2018-19 OVC 808 (1st)


In [246]:
specific_award = []
for i in awards_sub_rect:#
    specific_award.append(i.find_all('td',{'class':'single'}))


In [289]:
# player without awars
url = 'https://www.sports-reference.com/cbb/players/aaron-douglas-2.html'
response = requests.get(url)
html = response.text
html = html.replace('<!--','').replace('-->','')
soup = BeautifulSoup(html, 'lxml')

leaderboard = soup.find_all('div', {'id':re.compile('leaderboard_')})
if len(leaderboard)>0:
    print('do normal code')
else:
    print(id_ + '::' +'no_awards' +'::' +'no_awards')

temetrius-morant-1::no_awards::no_awards


In [287]:
len(leaderboard)

0

In [238]:
# list of awards
for i in x.find_all('div', {'id':re.compile('leaderboard_')})[0].find_all('td',{'class':'single'}):
    print(i.text)

2018-19 All-OVC - 1st Team
2018-19 Consensus All-America - 1st Team
2018-19 Naismith Award Finalists
2018-19 Naismith Award Semifinalists
2018-19 OVC Player of the Year
2018-19 The Bob Cousy Award
2018-19 USBWA Player of the Year Finalists
2018-19 Wooden Award - Finalists
2018-19 Wooden Award - Late Season
2018-19 Wooden Award - Midseason
2018-19 Wooden Award - National Ballot
2019 All-OVC Tournament - 1st Team
2019 OVC Tournament MVP


In [227]:

x = soup.find('div',{'class':'data_grid'})
print(x.find_all('div', {'id':re.compile('leaderboard_')})[2].prettify())

<div class="data_grid_box" id="leaderboard_pts_per_g">
 <table class="no_columns">
  <caption class="poptip" data-tip="Points Per Game">
   Points Per Game
  </caption>
  <tr class="">
   <td class="single">
    <a href="/cbb/conferences/ovc/2018-leaders.html">
     2017-18
    </a>
    <a href="/cbb/conferences/ovc/2018-leaders.html">
     OVC
    </a>
    12.7 (19th)
   </td>
  </tr>
  <tr class="">
   <td class="single">
    <a href="/cbb/seasons/2019-leaders.html">
     2018-19
    </a>
    <a href="/cbb/seasons/2019-leaders.html">
     NCAA
    </a>
    24.5 (7th)
   </td>
  </tr>
  <tr class="first_place">
   <td class="single">
    <a href="/cbb/conferences/ovc/2019-leaders.html">
     2018-19
    </a>
    <a href="/cbb/conferences/ovc/2019-leaders.html">
     OVC
    </a>
    24.5 (1st)
   </td>
  </tr>
 </table>
 <button onclick="vjs_addClass(document.getElementById('leaderboard_pts_per_g'),'show_all')">
  3 Seasons in Top 20
 </button>
</div>

